In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
url = "https://imsdb.com/scripts/Lord-of-the-Rings-The-Two-Towers.html#/"
response = requests.get(url)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

# Locate the main content section containing the script
main_content = soup.find('td', class_='scrtext')

# Dictionary to store the dialogues
dialogues = {'Gandalf': [], 'Other Character': []}

In [4]:
if main_content:
    # Find the <pre> tag within the main content
    script_content = main_content.find('pre')
    if script_content:
        # Remove content within square brackets
        script_text = re.sub(r'\[.*?\]', '', script_content.text, flags=re.DOTALL)
        lines = script_text.split('\n')
        
        current_character = None
        is_gandalf_speaking = False
        gandalf_line = ''
        other_character_line = ''
        other_character_name = ''

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # Check if the line is a character name (uppercase line)
            if line.isupper() and len(line) > 1:
                if current_character == 'GANDALF' and gandalf_line and other_character_line:
                    dialogues['Gandalf'].append(gandalf_line.strip())
                    dialogues['Other Character'].append(other_character_line.strip())
                    gandalf_line = ''
                    other_character_line = ''
                    other_character_name = ''

                current_character = line
                print(f"Character detected: {current_character}")  # Debug print

                # Check if Gandalf is the current speaker
                if current_character == 'GANDALF':
                    is_gandalf_speaking = True
                else:
                    is_gandalf_speaking = False
                    other_character_name = current_character
            elif current_character and line:
                # Add the line to the current character's lines
                if is_gandalf_speaking:
                    gandalf_line += ' ' + line
                else:
                    other_character_line += ' ' + line

        # Capture any remaining dialogue
        if current_character == 'GANDALF' and gandalf_line and other_character_line:
            dialogues['Gandalf'].append(gandalf_line.strip())
            dialogues['Other Character'].append(other_character_line.strip())

else:
    print("Could not find the main script content section.")

Character detected: THE LORD OF THE RINGS: THE TWO TOWERS
Character detected: GANDALF
Character detected: FRODO
Character detected: GANDALF
Character detected: GANDALF
Character detected: YOU� SHALL NOT... PASS!!!
Character detected: GANDALF
Character detected: BOROMIR
Character detected: FRODO
Character detected: GANDALF
Character detected: FRODO
Character detected: FRODO
Character detected: FRODO
Character detected: SAM
Character detected: FRODO
Character detected: SAM
Character detected: FRODO
Character detected: SAM
Character detected: FRODO
Character detected: FRODO
Character detected: SAM
Character detected: SAM
Character detected: FRODO
Character detected: SAM
Character detected: SAM
Character detected: FRODO
Character detected: SAM
Character detected: FRODO
Character detected: GOLLUM
Character detected: FRODO
Character detected: GOLLUM
Character detected: SAM
Character detected: SAM
Character detected: GOLLUM
Character detected: SAM
Character detected: FRODO
Character detected:

In [5]:
# Create a DataFrame from the dialogues dictionary
df = pd.DataFrame(dialogues)
df

,Gandalf,Other Character
0,You cannot pass!,"Screenplay by Peter Jackson, Fran Walsh and Ph..."
1,"I am a servant of the Secret Fire, wielder of ...",Gandalf!
2,Argh! Go back to the shadow. The Dark Fire wil...,No! No! Gandalf!
3,Through fire and water. From the lowest dunge...,Noooooooooooooooo!!!! Gaaandaaaaalf!! Gandalf!...
4,"Until at last, I threw down my enemy and smote...",Gandalf!
5,I am Gandalf the White. And I come back to yo...,Gandalf!
6,"One stage of your journey is over, another beg...","That is one of the Mearas, unless my eyes are ..."
7,Edoras and the Golden Hall of Meduseld. There ...,My home lies deep in the forest near the roots...
8,Be careful what you say. Do not look for welco...,"My lord, your son, he is dead. My lord? Uncle?..."
9,Ah.,"Oh, he� he must have died sometime in the nigh..."


In [6]:
df.to_csv('gandalf_test.csv', index=False)